# Overview

Make sure that this notebook is run in the Conda environment setup for this project, as described in [Readme.md](README.md)

This notebook runs the code for performing schooling experiments. 

# Parameters

In [1]:
# Prefix for video files
moviefile_prefix = "SCHOOL_S001_S001_T"

# Packages

In [2]:
from DMXEnttecPro import Controller
import numpy as np
import def_runexperiments as re
import multiprocess, os
# import pandas as pd

# This is necessary for using multiprocess on a Mac
if os.name == "posix":
        multiprocess.set_start_method('spawn')

## Project paths

You can find instructions for defining hw_address in [README.md](README.md)

In [3]:
# Address for the Enttec USB device (see above)
hw_address = "/dev/cu.usbserial-EN373474"

# Path to time-code audio file
aud_path = "/Users/experimentalist/Documents/Projects/wake_tracking/timecode.wav"

# Path to log csv file
log_path = "/Volumes/schooling/waketracking/recording_log.csv"

# Check address of log file
if not os.path.isfile(log_path):
    raise OSError("log_path not found at " + log_path)

# Check address of audio file
if not os.path.isfile(aud_path):
    raise OSError("aud_path not found at " + aud_path)

# log = pd.read_csv(log_path)
# print(log)

## Initialize hardware

Run these lines for all of the programs that you want to run below.

In [4]:
#  Initialize hardware
dmx = Controller(hw_address, auto_submit=True, dmx_size=32)

# Set output rate at maximum
dmx.set_dmx_parameters(output_rate=0)

# Basic hardware control

## Change fixed light intensity

In [10]:
# Specify light intensity (from 0 to 1)
light_intensity = 0

# Sets DMX channel 1 to max 255 (Channel 1 is the intensity)
dmx.set_channel(1, int(light_intensity*255))  

# Programs

Select which of the cells below that make sense for the experiment that you want to run.

## Lights on for fixed duration

In [31]:
# What light level to return to after the program
end_light_level = 0

# list of light levels
light_level = np.array([1])

# Duration that each level is fixed
light_dur = np.array([2])

# Run control program
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, light_dur=light_dur, trig_video=True, echo=False, plot_data=True, movie_prefix=moviefile_prefix)

# Set ending log intensity
dmx.set_channel(1, int(end_light_level*255))  

# print(df)

Experiment -- Date: 2022-12-15, Trial number: 8 ----------
    Starting audio to trigger video recording
    Timecode audio ended.
    Video will NOT be analyzed
    Video filename: SCHOOL_S001_S001_T014
    Log file saved to: /Volumes/schooling/waketracking/recording_log.csv


## Lights on -> Lights off

Ramp down light intensity at a constant rate.

In [8]:
# list of light levels
light_level = np.array([1, 0])

# Duration that each level is fixed
light_dur = np.array([1, 2])

# Duration for the changes in light level
ramp_dur = np.array([0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,movie_prefix=moviefile_prefix)

# print(df)

Starting audio to trigger video recording . . .
    Timecode audio ended.
    Video will NOT be analyzed
    Log file saved to: /Volumes/schooling/waketracking/recording_log.csv


## Lights off -> Lights on

Ramp down light intensity at a constant rate.

In [33]:
# list of light levels
light_level = np.array([0, 1])

# Duration that each level is fixed
light_dur = np.array([1, 2])

# Duration for the changes in light level
ramp_dur = np.array([2])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,movie_prefix=moviefile_prefix)

# print(df)

Starting audio to trigger video recording . . .
    Timecode audio ended.


## Ramp down and then up

Changing light intensity at a constant rate.

In [9]:
# list of light levels
light_level = np.array([1, 0, 0.5])

# Duration that each level is fixed
light_dur = np.array([1, 2, 1])

# Duration for the changes in light level
ramp_dur = np.array([0.5, 0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True, movie_prefix=moviefile_prefix)

# print(df)

Starting audio to trigger video recording . . .
    Timecode audio ended.
    Video will NOT be analyzed
    Log file saved to: /Volumes/schooling/waketracking/recording_log.csv
